In [1]:
# 網路上有人提到在使用pandas_datareader前加入這一行，果然可以
# ref: https://stackoverflow.com/questions/50394873/import-pandas-datareader-gives-importerror-cannot-import-name-is-list-like
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like

import ffn
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

In [2]:
from inspect import getmembers, isfunction
import all_strategies

In [3]:
# 計算 MaxDD
def DrawDownAnalysis(cumRet):
    dd_series = ffn.core.to_drawdown_series(cumRet)
    dd_details = ffn.core.drawdown_details(dd_series)
    return dd_details['drawdown'].min(), dd_details['days'].max()

In [4]:
# 利用策略產生的持有部位資訊，計算底下四個指標來判斷投資績效
# sharpe ratio: 判斷報酬的好壞跟穩定度，數值越大越好
# maxdd: maximum drawdown, 最糟糕的狀況會賠幾 %
# maxddd: maximum drawdown duration, 低於上一次最高報酬的天數
# cumRet[-1]: 最後賺的 % 數
def indicators(df):
    dailyRet = df['Close'].pct_change()
    excessRet = (dailyRet - 0.04/252)[df['positions'] == 1]
    SharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)

    cumRet = np.cumprod(1+excessRet)

    maxdd, maxddd = DrawDownAnalysis(cumRet)

    return SharpeRatio, maxdd, maxddd, cumRet[-1]

In [5]:
def apply_strategy(strategy, df):
    return strategy(df)

In [6]:
# 讀出預先下載好的股價資料
with open('../data/twstockdata', 'rb') as f:
    data = pickle.load(file=f)

# 計算各支股票的回測結果
results = []

strategies = [member[1] for member in getmembers(all_strategies) if isfunction(member[1])]

for symbol in data:
    for strategy in strategies:
        try:
            apply_strategy(strategy, data[symbol])
            if np.all(data[symbol]['signals']==0):
                print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                continue
            SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
            days = (data[symbol].index[-1] - data[symbol].index[0]).days
            results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                            data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol, strategy.__name__))
        except Exception as e:
            print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)


results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','days', 'entries','symbol','strategy'])

Symbol: 2330.tw 使用 BBands_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 LG_minus3_CL 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 RSI_7030_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 RSI_8020_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 team2_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 第一組_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 第六組_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 財運滾滾來_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 BBands_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 LG_minus3_CL 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 RSI_7030_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 RSI_8020_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 team2_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 第一組_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 第六組_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 財運滾滾來_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 BBands_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 LG_minus3_CL 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 RSI_7030_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 RSI_8020_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 team2_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 第一組_stra

In [7]:
# Sorted by MaxDrawDown:
results_df.sort_values('MaxDrawDown',ascending=False).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
6,1.156600,-0.124380,412,1.326028,965,20,2317.tw,Best_strategy
1,0.174257,-0.130429,350,1.016355,965,13,2330.tw,Breakout_strategy
12,-0.200408,-0.141440,769,0.956854,965,27,2412.tw,Best_strategy
13,-0.292994,-0.142588,769,0.950460,965,9,2412.tw,Breakout_strategy
5,0.952476,-0.163674,114,1.223382,965,8,2330.tw,大盜韓不住_strategy


In [8]:
# Sorted by returns:
results_df.sort_values('returns',ascending=False).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
21,1.731203,-0.558185,279,7.946098,965,12,3406.tw,Team3_strategy
23,1.815209,-0.563141,355,6.903885,965,56,3406.tw,中山南拳寶寶_strategy
20,1.435866,-0.612821,355,5.738168,965,1,3406.tw,JuianJuian4715_strategy
18,2.225839,-0.315184,252,5.225918,965,26,3406.tw,Best_strategy
19,2.509156,-0.300154,252,4.918509,965,11,3406.tw,Breakout_strategy


In [9]:
# Sorted by sharpe:
results_df.sort_values('sharpe',ascending=False).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
19,2.509156,-0.300154,252,4.918509,965,11,3406.tw,Breakout_strategy
18,2.225839,-0.315184,252,5.225918,965,26,3406.tw,Best_strategy
23,1.815209,-0.563141,355,6.903885,965,56,3406.tw,中山南拳寶寶_strategy
21,1.731203,-0.558185,279,7.946098,965,12,3406.tw,Team3_strategy
25,1.613078,-0.352816,370,2.158782,965,19,3008.tw,Best_strategy


In [10]:
# Sorted by MaxDrawDownDuration:
results_df.sort_values('MaxDrawDownDuration',ascending=True).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
5,0.952476,-0.163674,114,1.223382,965,8,2330.tw,大盜韓不住_strategy
2,0.859279,-0.215589,215,1.445836,965,1,2330.tw,JuianJuian4715_strategy
3,0.831572,-0.213549,215,1.410539,965,7,2330.tw,Team3_strategy
19,2.509156,-0.300154,252,4.918509,965,11,3406.tw,Breakout_strategy
18,2.225839,-0.315184,252,5.225918,965,26,3406.tw,Best_strategy
